In [1]:
import numpy as np
import csv
import pandas as pd
import os
import torch
import matplotlib.pyplot as plt

In [2]:
import camera_tools as ct

In [3]:
from cmac2 import CMAC

In [4]:
#Calibrate the camera to detect green box, if you haven't done this calibration before
low_green, high_green = ct.colorpicker()
print(low_green)
print(high_green)

colorpicker() -> Cannot read frame - ending execution...


UnboundLocalError: cannot access local variable 'v1_min' where it is not associated with a value

In [6]:
low_green = np.array([51, 70, 93])
high_green = np.array([120,103,208])

In [7]:
#Check whether the camera detects the green object properly
cam = ct.prepare_camera()
image = ct.capture_image(cam)
x,y = ct.locate(image, low_green, high_green)
ct.show_camera(cam)

print(x,y)

158 184


In [8]:
from FableAPI.fable_init import api
api.setup(blocking=True)

PORT: COM3, NAME: Standard Serial over Bluetooth link (COM3), HWID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_VID&0001005D_PID&223B\7&30FFFDFE&1&1C52169F0364_C00000000
PORT: COM4, NAME: Standard Serial over Bluetooth link (COM4), HWID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_LOCALMFG&0000\7&30FFFDFE&1&000000000000_00000000
PORT: COM5, NAME: USB Serial Device (COM5), HWID: USB VID:PID=03EB:FABE SER= LOCATION=1-1
Trying to connect COM5 Connection Exception  (<class 'serial.serialutil.SerialException'>, SerialException("could not open port 'COM5': PermissionError(13, 'Access is denied.', None, 5)"), <traceback object at 0x0000027645C66A40>)
...failure
PORT: COM3, NAME: Standard Serial over Bluetooth link (COM3), HWID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_VID&0001005D_PID&223B\7&30FFFDFE&1&1C52169F0364_C00000000
PORT: COM4, NAME: Standard Serial over Bluetooth link (COM4), HWID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_LOCALMFG&0000\7&30FFFDFE&1&000000000000_000

KeyboardInterrupt: 

PORT: COM3, NAME: Standard Serial over Bluetooth link (COM3), HWID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_VID&0001005D_PID&223B\7&30FFFDFE&1&1C52169F0364_C00000000
PORT: COM4, NAME: Standard Serial over Bluetooth link (COM4), HWID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_LOCALMFG&0000\7&30FFFDFE&1&000000000000_00000000
PORT: COM5, NAME: USB Serial Device (COM5), HWID: USB VID:PID=03EB:FABE SER= LOCATION=1-1
Trying to connect COM5 Connection Exception  (<class 'serial.serialutil.SerialException'>, SerialException("could not open port 'COM5': PermissionError(13, 'Access is denied.', None, 5)"), <traceback object at 0x000002764AF35B80>)
...failure
PORT: COM3, NAME: Standard Serial over Bluetooth link (COM3), HWID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_VID&0001005D_PID&223B\7&30FFFDFE&1&1C52169F0364_C00000000
PORT: COM4, NAME: Standard Serial over Bluetooth link (COM4), HWID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_LOCALMFG&0000\7&30FFFDFE&1&000000000000_000

In [9]:
moduleids = api.discoverModules()
print("Module IDs: ", moduleids)
moduleID = moduleids[0]
print("Battery level:",api.getBattery(moduleID),"%")

Module IDs:  None


TypeError: 'NoneType' object is not subscriptable

PORT: COM3, NAME: Standard Serial over Bluetooth link (COM3), HWID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_VID&0001005D_PID&223B\7&30FFFDFE&1&1C52169F0364_C00000000
PORT: COM4, NAME: Standard Serial over Bluetooth link (COM4), HWID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_LOCALMFG&0000\7&30FFFDFE&1&000000000000_00000000
PORT: COM5, NAME: USB Serial Device (COM5), HWID: USB VID:PID=03EB:FABE SER= LOCATION=1-1
Trying to connect COM5 Connection Exception  (<class 'serial.serialutil.SerialException'>, SerialException("could not open port 'COM5': PermissionError(13, 'Access is denied.', None, 5)"), <traceback object at 0x0000027646A32B80>)
...failure
PORT: COM3, NAME: Standard Serial over Bluetooth link (COM3), HWID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_VID&0001005D_PID&223B\7&30FFFDFE&1&1C52169F0364_C00000000
PORT: COM4, NAME: Standard Serial over Bluetooth link (COM4), HWID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_LOCALMFG&0000\7&30FFFDFE&1&000000000000_000

In [ ]:
A = 120
T = 200

In [ ]:
api.setPos(-90,0,moduleID)

In [ ]:
#Now integrate the CMAC to the previous control loop which had only the Neural Network. 
#The implementation of the CMAC can be found in code given for second week exercises.
#TODO: Implement the control loop with both neural network and CMAC. 
def ControlLoopWithBothNNandCMAC(target_x, target_y, iter_num):
    
    # api.setPos(-90,0,moduleID)
    target_x_coord = target_x
    target_y_coord = target_y

    x_ref_vec = []
    x_act_vec = []

    number_of_iterations_for_convergence = 0
    max_iter = iter_num

    ## TODO: CMAC initialization
    n_rfs = 1
    beta = 0.05

    xmin = [152,152]
    xmax = [448, 448]

    c = CMAC(n_rfs, xmin, xmax, beta)

    # P-controller
    Kp = 1

    ## Simulation loop
    for i in range(max_iter):
       
        cam = ct.prepare_camera()
        img = ct.capture_image(cam)
        x,y = ct.locate(img,low_green, high_green)

        # cam_red = ct.prepare_camera()
        # img_red = ct.capture_image(cam)
        # x_red,y_red = ct.locate(img,low_red, high_red)

        x_ref = A * np.sin(2*np.pi * i/T) + 300
        # x_ref = 450
        y_ref = y

        x_ref = float(x_ref)
        y_ref = float(y_ref)

        print(i, x, x_ref)

        x_ref_vec.append(x_ref)
        x_act_vec.append(x)

        # print(x,y)
        robot_x_coord_in_image = x
        robot_y_coord_in_image = y
        x_coord_error = x_ref - robot_x_coord_in_image;
        y_coord_error = y_ref - robot_y_coord_in_image;
        
        ## TODO: Implement the CMAC controller into the loop
        x_cmac = [x_ref ,robot_x_coord_in_image]
        angle_cmac = c.predict(x_cmac)

        # print(x_cmac)        
    
        #Here if the errors are less than twenty pixels we assume robot reaches the target. 
        # However you can choose any reasonable threshold value instead of 20.
        # if (np.abs(x_coord_error) < (1+x_prev*1.02) and np.abs(y_coord_error) < (1+y_coord_error*1.02)):
        #     print("number of iterations for convergence = ", number_of_iterations_for_convergence)
        #     break

        p_out = Kp * x_coord_error
       
        #TODO: Get the current Y (angular) position/angle of the robot. you can use the api.getPos function of fable
        robot_current_Y_pos = api.getPos(1,moduleID)
        #Next Y angular position of the robot will be robot_Y_pos + prediction_for_Y_pos_increment
        robot_next_Y_pos = robot_current_Y_pos + p_out
        
        c.learn(p_out)
        # print(angle_cmac)
        robot_nn_and_cmac = robot_next_Y_pos #+ angle_cmac

        #TODO: Set the next position of the robot to (-90,robot_next_Y_pos) using the setPos function of the fable.
        api.setPos(-90,robot_nn_and_cmac,moduleID)
        api.sleep(0.3)
        number_of_iterations_for_convergence = number_of_iterations_for_convergence + 1

    return [x_ref_vec,x_act_vec]

[x_ref_vec,x_act_vec] = ControlLoopWithBothNNandCMAC(0,0, 200)
plt.plot(np.arange(len(x_ref_vec)),x_ref_vec)
plt.plot(np.arange(len(x_act_vec)),x_act_vec)
plt.show()